In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from firefly.data_reader.GAIAreader import GaiaReader

palettable colormaps are not installed
don't have phil's colormaps


In [3]:
import os
import numpy as np

In [4]:

gaiadir = os.path.join(os.environ['HOME'],'projects','gaia','GaiaSource')
my_reader = GaiaReader(
    nthreads=52,
    write_startup=True,
    clean_datadir=True,
    use_mps=True,
    min_to_refine=1e5/10,
    nrecurse=4)

No work to be done! Celebrate!

3.2835262934366862 min elapsed
Make sure each field_array (3) has a field_filter_flag (0), assuming True.
Make sure each field_array (3) has a field_colormap_flag (0), assuming True.
Make sure each field_array (3) has a field_radius_flag (0), assuming False.
No work to be done! Celebrate!

2.3924498597780866 min elapsed
Make sure each field_array (3) has a field_filter_flag (0), assuming True.
Make sure each field_array (3) has a field_colormap_flag (0), assuming True.
Make sure each field_array (3) has a field_radius_flag (0), assuming False.
datadir: /home/abg6257/projects/gaia -- is not a sub-directory of firefly/static/data. 
This may produce confusing or inoperable results. As such, we will create a symlink for you when you writeToDisk.


In [6]:
from firefly.data_reader.settings import GUIurls

In [7]:
print(GUIurls)

['main', 'main/general', 'main/general/data', 'main/general/data/decimation', 'main/general/data/savepreset', 'main/general/data/reset', 'main/general/data/loadnewdata', 'main/general/camera', 'main/general/camera/centertextboxes', 'main/general/camera/cameratextboxes', 'main/general/camera/rotationtextboxes', 'main/general/camera/camerabuttons', 'main/general/camera/fullscreen', 'main/general/camera/snapshot', 'main/general/camera/camerafriction', 'main/general/camera/stereosep', 'main/general/projection', 'main/general/projection/columndensitycheckbox', 'main/general/projection/columndensitylogcheckbox', 'main/general/projection/columndensityselectcmap', 'main/general/projection/columndensitysliders', 'colorbarcontainer', 'fpscontainer', 'octreeloadingbarcontainer', 'main/particles']


In [ ]:
for pkey in my_reader.settings['sizeMult'].keys():
    my_reader.settings['sizeMult'][pkey] = 0.1
    
my_reader.settings['GUIExcludeList'] = ['colorbarcontainer']
my_reader.settings['collapseGUIAtStart'] = False
my_reader.settings['depthTest'] = False
my_reader.settings['blendingMode'] = 'additive'
my_reader.settings['showParts']['DR3-RV'] = False
my_reader.settings['camera'] = [0,0,0]
my_reader.settings['cameraRotation'] = [-1.5,0,-1.5]
my_reader.settings['startFly'] = True
my_reader.dumpSettingsToJSON()

('/home/abg6257/projects/gaia/GaiaDataSettings.json',
 '/home/abg6257/projects/gaia/GaiaDataSettings.json')

In [16]:
my_reader.writeToDisk(nthreads=52)

DR3-noRV - 966,113,753 parts (31,451 nodes) - 3 fields
DR3-RV - 31,431,382 parts (8,506 nodes) - 3 fields


''

In [23]:
my_reader.copyFireflySourceToTarget('/projects/b1094/ageller/Firefly_ABG/',dump_data=False,overwrite=False,nthreads=52)

['/projects/b1094/ageller/Firefly_ABG/']